In [ ]:
# default_exp __init__

# MXC API
https://mxcdevelop.github.io/APIDoc/

In [ ]:
# export
import contextlib
import hashlib
import hmac
import os
import requests
import time

from urllib import parse


class MXC(object):

    ROOT_URL = 'https://www.mxc.ceo'
    ERROR_MESSAGES = {
        400: '请求参数无效。',
        401: '签名验证失败。',
        429: '请求过于频繁。',
        10072: '无效的 Access Key。',
        10073: '无效的请求时间。',
        30000: '当前交易对已暂停交易。',
        30001: '当前交易方向不允许下单。',
        30002: '小于最小交易金额。',
        30003: '大于最大交易金额。',
        30004: '持仓不足。',
        30005: '卖出超额。',
        30010: '下单价格超过范围。',
        30016: '暂停交易。',
        30019: '订单集合超出最大允许长度。',
        30020: '受限制的交易对，暂不支持 API 访问。',
        30021: '无效的交易对。',
    }

    def __init__(self, API_KEY, SECRET_KEY):
        self.API_KEY = API_KEY
        self.SECRET_KEY = SECRET_KEY

    @staticmethod
    def _get_server_time():
        return int(time.time())

    def _sign(self, method, path, original_params=None):
        params = {
            'api_key': self.API_KEY,
            'req_time': type(self)._get_server_time(),
        }
        if original_params is not None:
            params.update(original_params)
        params_str = '&'.join('{}={}'.format(k, params[k]) for k in sorted(params))
        to_sign = '\n'.join([method, path, params_str])
        params.update({
            'sign': hmac.new(self.SECRET_KEY.encode(), to_sign.encode(), hashlib.sha256).hexdigest()
        })
        return params

    def _request(self, method, path, params):
        url = '{}{}'.format(type(self).ROOT_URL, path)
        with contextlib.closing(
            requests.request(method, url, params=params)
        ) as response:
            response_json = response.json()
            code = response_json.get('code', 0)
            if code != 200:
                raise Exception(
                    type(self).ERROR_MESSAGES.get(code, response_json.get('msg', '未知错误。'))
                )
            return response_json.get('data', {})

    def get_symbols(self):
        """marget data"""
        method = 'GET'
        path = '/open/api/v2/market/symbols'
        params = {'api_key': self.API_KEY}
        return self._request(method, path, params)

    def get_rate_limit(self):
        """rate limit"""
        method = 'GET'
        path = '/open/api/v2/common/rate_limit'
        params = {'api_key': self.API_KEY}
        return self._request(method, path, params)

    def get_timestamp(self):
        """get current time"""
        method = 'GET'
        path = '/open/api/v2/common/timestamp'
        params = {'api_key': self.API_KEY}
        return self._request(method, path, params)

    def get_ticker(self, symbol):
        """get ticker information"""
        method = 'GET'
        path = '/open/api/v2/market/ticker'
        params = {
            'api_key': self.API_KEY,
            'symbol': symbol,
        }
        return self._request(method, path, params)

    def get_depth(self, symbol, depth):
        """get market depth"""
        method = 'GET'
        path = '/open/api/v2/market/depth'
        params = {
            'api_key': self.API_KEY,
            'symbol': symbol,
            'depth': depth,
        }
        return self._request(method, path, params)

    def get_deals(self, symbol, limit):
        """get deals records"""
        method = 'GET'
        path = '/open/api/v2/market/deals'
        params = {
            'api_key': self.API_KEY,
            'symbol': symbol,
            'limit': limit,
        }
        return self._request(method, path, params)

    def get_kline(self, symbol, interval):
        """k-line data"""
        method = 'GET'
        path = '/open/api/v2/market/kline'
        params = {
            'api_key': self.API_KEY,
            'symbol': symbol,
            'interval': interval,
        }
        return self._request(method, path, params)

    def get_account_info(self):
        """account information"""
        method = 'GET'
        path = '/open/api/v2/account/info'
        params = self._sign(method, path)
        return self._request(method, path, params)

    def _request_with_json(self, method, path, params, json):
        url = '{}{}'.format(type(self).ROOT_URL, path)
        with contextlib.closing(
            requests.request(method, url, params=params, json=json)
        ) as response:
            response_json = response.json()
            code = response_json.get('code', 0)
            if code != 200:
                raise Exception(
                    type(self).ERROR_MESSAGES.get(code, response_json.get('msg', '未知错误。'))
                )
            return response_json.get('data', {})

    def place_order(self, symbol, price, quantity, trade_type, order_type):
        """place order"""
        method = 'POST'
        path = '/open/api/v2/order/place'
        params = self._sign(method, path)
        data = {
            'symbol': symbol,
            'price': price,
            'quantity': quantity,
            'trade_type': trade_type,
            'order_type': order_type,
        }
        return self._request_with_json(method, path, params, data)

    def batch_orders(self, orders):
        """batch order"""
        method = 'POST'
        path = '/open/api/v2/order/place_batch'
        params = self._sign(method, path)
        return self._request_with_json(method, path, params, orders)

    def cancel_order(self, order_id):
        """cancel in batch"""
        method = 'DELETE'
        path = '/open/api/v2/order/cancel'

        origin_trade_no = order_id
        if isinstance(order_id, list):
            origin_trade_no = parse.quote(','.join(order_id))
        params = self._sign(method, path, original_params={'order_ids': origin_trade_no})
        if isinstance(order_id, list):
            params['order_ids'] = ','.join(order_id)

        return self._request(method, path, params)

    def get_open_orders(self, symbol):
        """current orders"""
        method = 'GET'
        path = '/open/api/v2/order/open_orders'

        original_params = {
            'symbol': symbol,
        }
        params = self._sign(method, path, original_params=original_params)

        return self._request(method, path, params)

    def get_all_orders(self, symbol, trade_type):
        """order list"""
        method = 'GET'
        path = '/open/api/v2/order/list'

        original_params = {
            'symbol': symbol,
            'trade_type': trade_type,
        }
        params = self._sign(method, path, original_params=original_params)

        return self._request(method, path, params)

    def query_order(self, order_id):
        """query order"""
        method = 'GET'
        path = '/open/api/v2/order/query'

        origin_trade_no = order_id
        if isinstance(order_id, list):
            origin_trade_no = parse.quote(','.join(order_id))
        original_params = {
            'order_ids': origin_trade_no,
        }
        params = self._sign(method, path, original_params=original_params)
        if isinstance(order_id, list):
            params['order_ids'] = ','.join(order_id)

        return self._request(method, path, params)

    def get_deal_orders(self, symbol):
        """account deal records"""
        method = 'GET'
        path = '/open/api/v2/order/deals'

        original_params = {
            'symbol': symbol,
        }
        params = self._sign(method, path, original_params=original_params)

        return self._request(method, path, params)

    def get_deal_detail(self, order_id):
        """deal detail"""
        method = 'GET'
        path = '/open/api/v2/order/deal_detail'

        original_params = {
            'order_id': order_id,
        }
        params = self._sign(method, path, original_params=original_params)

        return self._request(method, path, params)

In [ ]:
# hide
import os
notebooks_dir = os.getcwd()
project_dir = os.path.dirname(notebooks_dir)

import sys
sys.path.append(project_dir)

In [ ]:
from ccstabilizer import secrets

In [ ]:
MXC(os.environ['MXC_ACCESS_KEY'], os.environ['MXC_SECRET_KEY']).get_deal_orders('AR_USDT')

[{'id': 'e38b467d1bdc4b0cbd95dbc460dfc914',
  'symbol': 'AR_USDT',
  'quantity': '0.23',
  'price': '22.259',
  'amount': '5.11957',
  'fee': '0.01023914',
  'trade_type': 'ASK',
  'order_id': '57fb12e86abf426eb1a2847a3fd14a8a',
  'is_taker': True,
  'fee_currency': 'USDT',
  'create_time': 1616222096000},
 {'id': '00ad88da35c54955bfcd020a1c75b340',
  'symbol': 'AR_USDT',
  'quantity': '9',
  'price': '22',
  'amount': '198',
  'fee': '0.396',
  'trade_type': 'BID',
  'order_id': 'a8678f5d008b4cfb8ab6eed01ff7d2d3',
  'is_taker': True,
  'fee_currency': 'USDT',
  'create_time': 1616221611000},
 {'id': '228761d1dfa44b609e18a73083c20497',
  'symbol': 'AR_USDT',
  'quantity': '1.85',
  'price': '21.966',
  'amount': '40.6371',
  'fee': '0.0812742',
  'trade_type': 'ASK',
  'order_id': '88b5b2a2287649a1b61d93cc272bb263',
  'is_taker': True,
  'fee_currency': 'USDT',
  'create_time': 1616221579000},
 {'id': '284c321ef8654378b2321aa380677df4',
  'symbol': 'AR_USDT',
  'quantity': '2.18',
  'p